# Data Preprocessing for Power BI

In [25]:
# Create output directory
import os
import pandas as pd
output_folder = "../data/PBI/MINDsmall_csv"
os.makedirs(output_folder, exist_ok=True)

tsv_files=["../data/raw/MINDsmall_dev/behaviors.tsv", "../data/raw/MINDsmall_dev/news.tsv"]
vec_files=["../data/raw/MINDsmall_dev/relation_embedding.vec", "../data/raw/MINDsmall_dev/entity_embedding.vec"]

# Function to read a file, take first 50 rows, and save it
def process_file(file_path, output_folder, sep="\t"):
    # Read the file
    df = pd.read_csv(file_path, sep=sep, header=None)
    
    # Save to new file in preview_50 folder
    output_filename = f"{os.path.basename(file_path).split('.')[0]}.csv"
    output_path = os.path.join(output_folder, output_filename)
    df.to_csv(output_path, index=False, header=False)
    
    print(f"Saved: {output_path}")

# Process the .tsv files
for file in tsv_files:
    process_file(file, output_folder, sep="\t")

# Process the .vec files (also tab-separated)
for file in vec_files:
    process_file(file, output_folder, sep="\t")

print("Preview files created successfully in 'MINDsmall_csv' folder.")

Saved: ../data/PBI/MINDsmall_csv/behaviors.csv
Saved: ../data/PBI/MINDsmall_csv/news.csv
Saved: ../data/PBI/MINDsmall_csv/relation_embedding.csv
Saved: ../data/PBI/MINDsmall_csv/entity_embedding.csv
Preview files created successfully in 'MINDsmall_csv' folder.


In [ ]:
import pandas as pd
import os
import ast

# Create data folder
data_folder = "star_schema_data"
os.makedirs(data_folder, exist_ok=True)

# Load datasets with correct column names
behaviors_columns = ["Impression ID", "User ID", "Time", "History", "Impressions"]
news_columns = ["News ID", "Category", "SubCategory", "Title", "Abstract", "URL", "Title Entities", "Abstract Entities"]
entity_embedding_columns = ["Entity ID"] + [f"Embedding_{i}" for i in range(1, 101)]
relation_embedding_columns = ["Relation ID"] + [f"Embedding_{i}" for i in range(1, 101)]

# Load datasets
behaviors_df = pd.read_csv("../data/PBI/MINDsmall_csv/behaviors.csv", names=behaviors_columns, header=None)
news_df = pd.read_csv("../data/PBI/MINDsmall_csv/news.csv", names=news_columns, header=None)
entity_embedding_df = pd.read_csv("../data/PBI/MINDsmall_csv/entity_embedding.csv", names=entity_embedding_columns, header=None)
relation_embedding_df = pd.read_csv("../data/PBI/MINDsmall_csv/relation_embedding.csv", names=relation_embedding_columns, header=None)

# Convert time format
behaviors_df["Time"] = pd.to_datetime(behaviors_df["Time"], format="%m/%d/%Y %I:%M:%S %p", errors='coerce')

# Process entity columns
news_df["Title Entities"] = news_df["Title Entities"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])
news_df["Abstract Entities"] = news_df["Abstract Entities"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

def extract_entity_info(entity_list):
    if not entity_list:
        return None, None, None, None
    entity = entity_list[0]  # Take the first entity if multiple exist
    return entity.get("Label"), entity.get("Type"), entity.get("WikidataId"), entity.get("Confidence", None)

news_df[["Entity Label", "Entity Type", "Wikidata ID", "Confidence Score"]] = news_df["Title Entities"].apply(
    lambda x: pd.Series(extract_entity_info(x))
)

# Ensure Dim_Entities contains correct entity metadata
if "Confidence Score" in entity_embedding_df.columns:
    dim_entities = entity_embedding_df[["Entity ID", "Label", "Type", "Wikidata ID", "Confidence Score"]]
else:
    dim_entities = entity_embedding_df[["Entity ID"]]  # If Confidence Score is missing, default to Entity ID only

dim_entities.to_csv(os.path.join(data_folder, "Dim_Entities.csv"), index=False)

# Compute Aggregated Click Behavior and Engagement Score for Dim_Users
user_click_counts = behaviors_df["User ID"].value_counts().reset_index()
user_click_counts.columns = ["User ID", "Aggregated Click Behavior"]
user_click_counts["Engagement Score"] = user_click_counts["Aggregated Click Behavior"] / user_click_counts["Aggregated Click Behavior"].max()

dim_users = behaviors_df[["User ID"]].drop_duplicates().merge(user_click_counts, on="User ID", how="left")

# Create and save tables
fact_impressions = behaviors_df[["Impression ID", "User ID", "Time", "History", "Impressions"]]
fact_impressions.to_csv(os.path.join(data_folder, "Fact_Impressions.csv"), index=False)

dim_users.to_csv(os.path.join(data_folder, "Dim_Users.csv"), index=False)

dim_news = news_df[["News ID", "Category", "SubCategory", "Title", "Abstract", "Entity Label", "Entity Type", "Wikidata ID", "Confidence Score"]]
dim_news.to_csv(os.path.join(data_folder, "Dim_News.csv"), index=False)

dim_relations = relation_embedding_df[["Relation ID"] + [f"Embedding_{i}" for i in range(1, 101)]]
dim_relations.to_csv(os.path.join(data_folder, "Dim_Relations.csv"), index=False)

# Display tables
for name, df in zip(["Fact_Impressions", "Dim_Users", "Dim_News", "Dim_Entities", "Dim_Relations"], 
                     [fact_impressions, dim_users, dim_news, dim_entities, dim_relations]):
    print(f"{name} Table:")
    print(df.head(), "\n")


KeyError: 'Confidence Score'